# Neural network implementation

In [1]:
import numpy as np
import os

from fault_tolerant_ml.data.mnist import MNist
from fault_tolerant_ml.ml import nn
import fault_tolerant_ml.ml.nn.activation as F

%reload_ext autoreload
%autoreload 2

## Read in data

In [2]:
data_dir = "../data"
filepaths = {
    "train": {
        "images": os.path.join(data_dir, "train-images-idx3-ubyte.gz"), "labels": os.path.join(data_dir, "train-labels-idx1-ubyte.gz")
    },
    "test": {
        "images": os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "labels": os.path.join(data_dir, "t10k-labels-idx1-ubyte.gz")
    }
}
mnist = MNist(filepaths)

In [3]:
mnist

<MNist X_train=(60000, 784), y_train=(60000, 10), X_test=(10000, 784), y_test=(10000, 10)>

In [4]:
n_features, n_classes = mnist.X_train.shape[1], mnist.y_train.shape[1]
print(f"n_features={n_features}, n_classes={n_classes}")

n_features=784, n_classes=10


In [5]:
theta = np.random.randn(n_features, n_classes)

In [115]:
class NeuralNet(nn.Model):
    
    def __init__(self):
        
        self.fc1 = nn.Layer(n_inputs=784, n_outputs=128)
        self.fc2 = nn.Layer(n_inputs=128, n_outputs=10)
        
        self.act_fn = F.Sigmoid()
    
    def forward(self, x):
        
        z1 = self.fc1(x)
        a1 = self.act_fn(z1)
        z2 = self.fc2(a1)
        y_pred = self.act_fn(z2)
        
        return y_pred, z2, a1, z1
    
    def backward(self, x, y, a_n):
        
        y_pred, z2, a1, z1 = a_n
        # Output layer error
        delta2 = (y_pred - y)# * self.act_fn.grad(z2)
        # Gradient of cost function
        dw2 = np.dot(a1.T, delta2)
        # Backpropagate the error through the network
        delta1 = np.dot(delta2, self.fc2.W.T) * self.act_fn.grad(z1)
        # Calculate gradient
        dw1 = np.dot(x.T, delta1)
        # Gradient of biases equal to the error
        db2 = np.sum(delta2, axis=0, keepdims=True)
        db1 = np.sum(delta1, axis=0, keepdims=True)
        return dw2, db2, dw1, db1

In [116]:
def cross_entropy_loss(y_pred, y):
    return np.mean(-y * np.log(y_pred) - (1 - y) * np.log(1 - y_pred))

In [117]:
def accuracy_score(y, y_pred):
    y_pred_ = y_pred.argmax(axis=1)
    y_ = y.argmax(axis=1)
    return np.sum(y_pred_==y_) / y_.shape[0]

In [114]:
model = NeuralNet()
print(model.fc1.shape)
print(model.fc2.shape)
epochs = 400
learning_rate = 0.99
m = mnist.X_train.shape[0]
for epoch in np.arange(epochs):
    
    # Feedforward
    y_pred, z2, a1, z1 = model.forward(mnist.X_train)
    
    # Calculate cost
    loss = cross_entropy_loss(y_pred, mnist.y_train)
    
    # Backprop
    dw2, db2, dw1, db1 = model.backward(mnist.X_train, mnist.y_train, [y_pred, z2, a1, z1])
    
    # Update weights
    model.fc2.W = model.fc2.W - learning_rate * 1 / m * dw2
    model.fc1.W = model.fc1.W - learning_rate * 1 / m * dw1
    model.fc2.b = model.fc2.b - learning_rate * 1 / m * db2
    model.fc1.b = model.fc1.b - learning_rate * 1 / m * db1
    
    acc = accuracy_score(mnist.y_train, y_pred)
    if epoch % 10 == 0:
        print(f'epoch = {epoch}, loss = {loss:.3f}, TRAIN ACC = {acc:.3f}')
    epoch += 1
    

(784, 128)
(128, 10)
epoch = 0, loss = 0.691, TRAIN ACC = 0.097
epoch = 10, loss = 0.345, TRAIN ACC = 0.112
epoch = 20, loss = 0.342, TRAIN ACC = 0.112
epoch = 30, loss = 0.342, TRAIN ACC = 0.112
epoch = 40, loss = 0.341, TRAIN ACC = 0.112
epoch = 50, loss = 0.341, TRAIN ACC = 0.112
epoch = 60, loss = 0.341, TRAIN ACC = 0.112
epoch = 70, loss = 0.341, TRAIN ACC = 0.114
epoch = 80, loss = 0.340, TRAIN ACC = 0.132
epoch = 90, loss = 0.339, TRAIN ACC = 0.160
epoch = 100, loss = 0.339, TRAIN ACC = 0.183
epoch = 110, loss = 0.338, TRAIN ACC = 0.200
epoch = 120, loss = 0.337, TRAIN ACC = 0.211
epoch = 130, loss = 0.335, TRAIN ACC = 0.219
epoch = 140, loss = 0.333, TRAIN ACC = 0.225
epoch = 150, loss = 0.331, TRAIN ACC = 0.228
epoch = 160, loss = 0.327, TRAIN ACC = 0.234
epoch = 170, loss = 0.323, TRAIN ACC = 0.243
epoch = 180, loss = 0.318, TRAIN ACC = 0.255
epoch = 190, loss = 0.312, TRAIN ACC = 0.272
epoch = 200, loss = 0.306, TRAIN ACC = 0.304
epoch = 210, loss = 0.299, TRAIN ACC = 0.358
